In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv('./data/kids.csv')

In [ ]:
pd.set_option("display.max_columns",None)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['model'].unique().shape

In [ ]:
data['subcategory'].unique()

In [ ]:
row_subcategory_drop = ['Culottes de grossesse', 'Chaussures pour garçon',
       'Brassières de grossesse',
       'Costumes pour bébé', 'Salopettes & Combinaisons',
       'Maillots de bain', 'Coutumes & Co-ords',
       'Costume & Jupe-culotte',
       'Blousons & Vestes', 'Pantalons', 'Chaussures pour fille',
       'Tops & Tees', 'Vêtements de nuit',
       'Sous-vêtements de maintien de grossesse',
       'Bas', 'Chaussures pour bébé',
       'Chapeaux', 'Pantalons & Capris',
       "Vêtements d'extérieur & Manteaux",
       'Robes & Sacs de couchage', 'Chaussures pour enfant',
       'Lingerie de grossesse', 'Accessoires pour cheveux', 'Sacs à dos',
       'Accesssoires des vêtements', 'Écharpes', 'Chaussettes',
       'Sets de coutumes', 'Lunettes de soleil', 'Sacs des couches',
       'Sacs à main', 'Vêtements pour fille',
       'Vêtements de nuit & Robes',
       'Colliers & Pendentifs', 'Imperméables', 'Accessoires pour enfant']

In [ ]:
row_subcategory_drop

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
drop_cloumns = ['discount', 'currency', 'brand', 'brand_url', 'codCountry', 'variation_0_color', 'variation_1_color', 'variation_0_thumbnail', 'variation_0_image', 'variation_1_thumbnail', 'variation_1_image', 'image_url', 'model', 'url']

In [ ]:
df = data.drop(drop_cloumns, axis=1)

In [ ]:
df.head()

In [ ]:
df['subcategory'].unique()

In [ ]:
for i in range(len(row_subcategory_drop)):
    index_names = df[ df['subcategory'] == row_subcategory_drop[i] ].index
    df.drop(index_names, inplace = True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
likes_features_df = df.pivot_table(index='name', columns='subcategory', values=['likes_count','current_price', 'is_new', 'raw_price']).fillna(0)
likes_features_df.to_csv('likes_features_df_kids.csv', index=False)
likes_features_df.head()

In [ ]:
likes_features_df.shape

In [ ]:
from scipy.sparse import csr_matrix

likes_features_df_matrix = csr_matrix(likes_features_df.values)

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
model_kids_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_kids_knn.fit(likes_features_df_matrix)

In [ ]:
query_index = np.random.choice(likes_features_df.shape[0])
print(query_index)

In [ ]:
likes_features_df.index

In [ ]:
query_index = 0
for i in range(likes_features_df.index.shape[0]):
    if likes_features_df.index[i] == '1Pc Belle Épingle à Cheveux Symbole Point D&#39;interrogation De Chapeaux Point D&#39;Exclamation Flèche ':
        query_index = i
        
query_index

In [ ]:
likes_features_df.iloc[query_index, :]

In [ ]:
distances, indices = model_kids_knn.kneighbors(likes_features_df.iloc[query_index, :].values.reshape(1,-1), n_neighbors=11)

In [ ]:
ls =[]
for i in range(0, len(distances.flatten())):
    if i == 0:
        print("Recommendations for {0}".format(likes_features_df.index[query_index]))
    else:
        print("{0}: {1}, with distance of {2}".format(i, likes_features_df.index[indices.flatten()[i]], distances.flatten()[i]))
        ls.append(likes_features_df.index[indices.flatten()[i]])

In [ ]:
likes_features_df.index[indices.flatten()[7]]

In [ ]:
ls

In [ ]:
new = data['name'].isin([likes_features_df.index[indices.flatten()[7]]])

In [ ]:
new_ls = data['name'].isin(ls)

In [ ]:
data[new_ls]

In [ ]:
import pickle

In [ ]:
pickle.dump(model_kids_knn, open('kids_model.pkl', 'wb'))